In [1]:
from sklearn import datasets, svm
iris = datasets.load_iris()

In [5]:
import pandas as pd
df = pd.DataFrame(iris.data, columns = iris.feature_names)
df['flower'] = iris.target
df['flower'] = df['flower'].apply(lambda x : iris.target_names[x])
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),flower
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [24]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.3)

In [25]:
model = svm.SVC(kernel = 'rbf', C=30, gamma='auto')
model.fit(X_train, y_train)
model.score(X_test,y_test)

0.9777777777777777

In [26]:
from sklearn.model_selection import cross_val_score

In [29]:
cross_val_score(svm.SVC(kernel='linear', C = 10, gamma='auto'),iris.data, iris.target, cv = 5)

array([1.        , 1.        , 0.9       , 0.96666667, 1.        ])

In [30]:
cross_val_score(svm.SVC(kernel='rbf', C = 10, gamma='auto'),iris.data, iris.target, cv = 5)

array([0.96666667, 1.        , 0.96666667, 0.96666667, 1.        ])

In [31]:
cross_val_score(svm.SVC(kernel='rbf', C = 20, gamma='auto'),iris.data, iris.target, cv = 5)

array([0.96666667, 1.        , 0.9       , 0.96666667, 1.        ])

In [34]:
import numpy as np
kernels = ['rbf', 'linear']
C = [1, 10, 20]
avg_scores = {}
for kval in kernels:
    for cval in C:
        cv_scores = cross_val_score(svm.SVC(kernel=kval, C = cval, gamma='auto'),iris.data, iris.target, cv = 5)
        avg_scores[kval + '_' + str(cval)] = np.average(cv_scores)
avg_scores

{'rbf_1': 0.9800000000000001,
 'rbf_10': 0.9800000000000001,
 'rbf_20': 0.9666666666666668,
 'linear_1': 0.9800000000000001,
 'linear_10': 0.9733333333333334,
 'linear_20': 0.9666666666666666}

In [50]:
from sklearn.model_selection import GridSearchCV
clf = GridSearchCV(svm.SVC(gamma = 'auto'),{
    'C':[1,10,20],
    'kernel':['rbf','linear']
}, cv = 5, return_train_score = False)
clf.fit(iris.data, iris.target)
clf.cv_results_

{'mean_fit_time': array([0.00099769, 0.00059857, 0.00100288, 0.00079193, 0.00098214,
        0.00059905]),
 'std_fit_time': array([8.34124359e-07, 4.88733514e-04, 1.14028904e-05, 3.96171985e-04,
        2.86755611e-05, 4.89123547e-04]),
 'mean_score_time': array([0.00059805, 0.00039878, 0.00059905, 0.00039935, 0.00040817,
        0.00060945]),
 'std_score_time': array([0.00048831, 0.0004884 , 0.00048912, 0.0004891 , 0.0004816 ,
        0.00049771]),
 'param_C': masked_array(data=[1, 1, 10, 10, 20, 20],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_kernel': masked_array(data=['rbf', 'linear', 'rbf', 'linear', 'rbf', 'linear'],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'C': 1, 'kernel': 'rbf'},
  {'C': 1, 'kernel': 'linear'},
  {'C': 10, 'kernel': 'rbf'},
  {'C': 10, 'kernel': 'linear'},
  {'C': 20, 'kernel': 'rbf'},
  {'C': 20

In [52]:
df = pd.DataFrame(clf.cv_results_)
df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.000998,8.341244e-07,0.000598,0.000488,1,rbf,"{'C': 1, 'kernel': 'rbf'}",0.966667,1.0,0.966667,0.966667,1.0,0.980000,0.016330,1
1,0.000599,4.887335e-04,0.000399,0.000488,1,linear,"{'C': 1, 'kernel': 'linear'}",0.966667,1.0,0.966667,0.966667,1.0,0.980000,0.016330,1
2,0.001003,1.140289e-05,0.000599,0.000489,10,rbf,"{'C': 10, 'kernel': 'rbf'}",0.966667,1.0,0.966667,0.966667,1.0,0.980000,0.016330,1
3,0.000792,3.961720e-04,0.000399,0.000489,10,linear,"{'C': 10, 'kernel': 'linear'}",1.000000,1.0,0.900000,0.966667,1.0,0.973333,0.038873,4
4,0.000982,2.867556e-05,0.000408,0.000482,20,rbf,"{'C': 20, 'kernel': 'rbf'}",0.966667,1.0,0.900000,0.966667,1.0,0.966667,0.036515,5
5,0.000599,4.891235e-04,0.000609,0.000498,20,linear,"{'C': 20, 'kernel': 'linear'}",1.000000,1.0,0.900000,0.933333,1.0,0.966667,0.042164,6


In [53]:
df[['param_C', 'param_kernel', 'mean_test_score']]

,param_C,param_kernel,mean_test_score
0,1,rbf,0.980000
1,1,linear,0.980000
2,10,rbf,0.980000
3,10,linear,0.973333
4,20,rbf,0.966667
5,20,linear,0.966667


In [54]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import svm

In [61]:
model_params = {
    'svm': {
        'model': svm.SVC(gamma='auto'),
        'params' : {
            'C': [1,10,20],
            'kernel': ['rbf','linear']
        }  
    },
    'random_forest': {
        'model': RandomForestClassifier(),
        'params' : {
            'n_estimators': [1,5,10]
        }
    },
    'logistic_regression' : {
        'model': LogisticRegression(solver='liblinear',multi_class='auto'),
        'params': {
            'C': [1,5,10]
        }
    }
}

In [62]:
scores = []

for model_name, mp in model_params.items():
    clf =  GridSearchCV(mp['model'], mp['params'], cv=5, return_train_score=False)
    clf.fit(iris.data, iris.target)
    scores.append({
        'model': model_name,
        'best_score': clf.best_score_,
        'best_params': clf.best_params_
    })
    
df = pd.DataFrame(scores,columns=['model','best_score','best_params'])
df

,model,best_score,best_params
0,svm,0.980000,"{'C': 1, 'kernel': 'rbf'}"
1,random_forest,0.953333,{'n_estimators': 10}
2,logistic_regression,0.966667,{'C': 5}
